In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import pandas as pd
import torch.nn.functional as F
import warnings
from pyplnoise import PinkNoise
warnings.filterwarnings('ignore')

if torch.cuda.is_available():
    torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



X = np.load('X.npy')
Y = np.load('Y.npy')

X_noise = np.empty([1256,19,30000])
noise_gen = PinkNoise(250, 1, 100)

for i in range(1256):
    for j in range(19):
        coef = np.mean(X[i,j,:])*0.01
        X_noise[i,j,:] = X[i,j,:] + coef * noise_gen.get_series(30000).T


X_augmented = np.vstack([X, X_noise])
Y_augmented = np.concatenate([Y, Y], axis=0)




class GraphConvolution(nn.Module):
    """
    Simple GCN layer.
    """
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def forward(self, input, adj):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return x


adj = pd.read_excel("D:\\SJN\model 2.0\parameter\con1_matrix.xlsx", engine='openpyxl',header=None)  # 邻接矩阵
adj = np.array(adj)
#adj = np.random.randn(19,19)+2
adj = torch.tensor(adj, dtype=torch.float32).to(device)


# 数据标准化
X_mean = X_augmented.mean()
X_std = X_augmented.std()
X_normalized = (X_augmented - X_mean) / X_std


#Y_mean = Y_augmented.mean()
#Y_std = Y_augmented.std()
Y_normalized = Y_augmented 

# 定义模型

class CombinedGCNCNN(nn.Module):
    def __init__(self, adj):
        super(CombinedGCNCNN, self).__init__()
        
        nfeat = 30000  # Initial feature number (equal to the sequence length)
        nhid = 100     # Number of hidden units
        nclass = 30000 # Output feature number (can be kept same as nfeat for simplicity)
        dropout = 0.3  # Dropout rate

        self.gcn = GCN(nfeat=nfeat, nhid=nhid, nclass=nclass, dropout=dropout)
        self.adj = adj
        
        # Your original CNN model
        self.conv1 = nn.Conv1d(19, 32, kernel_size=5, stride=2, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=2, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc1 = nn.Linear(128 * (30000 // 8), 256) # Updated the size due to stride=2 in all conv layers
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        
        # Transformer Layers
        self.embedding_dim = 128
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=self.embedding_dim, nhead=4, dim_feedforward=256, dropout=0.2),
            num_layers=2)
        
    def forward(self, x, adj):
        # GCN
        B, C, L = x.shape
        gcn_x = x.view(B, L, C).permute(0, 2, 1) # Reshape to [batch_size, num_nodes, num_features]
        gcn_out = self.gcn(gcn_x, self.adj)
        gcn_out = gcn_out.permute(0, 2, 1).reshape(B, C, L) # Reshape back to [batch_size, num_channels, seq_len]
        
        # Combine
        enhanced_x = x + gcn_out

        # Pass through CNN
        x = self.bn1(self.relu(self.conv1(enhanced_x)))
        x = self.bn2(self.relu(self.conv2(x)))
        x = self.bn3(self.relu(self.conv3(x)))
        # Reshape for Transformer
        x = x.permute(0, 2, 1)  # Transformer expects [batch, seq_len, features]
        
        # Pass through Transformer Encoder
        x = self.transformer_encoder(x)
        
        # Flatten for the FC layers
        x = x.permute(0, 2, 1)  
        x = x.reshape(x.size(0), -1)
        
        x = self.bn_fc1(self.relu(self.fc1(x)))
        x = self.fc2(x)

        return x

# When defining the optimizer, include weight decay for L2 regularization

# 数据加载
dataset = TensorDataset(torch.tensor(X_normalized, dtype=torch.float32), torch.tensor(Y_normalized[:, None], dtype=torch.float32))
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# 模型、优化器和损失函数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CombinedGCNCNN(adj=adj).to(device)
criterion_mse = nn.MSELoss() # 使用MSE作为损失函数
criterion_mae = nn.L1Loss()  # 使用L1Loss来计算MAE
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01) 

# 学习率调度器
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

# 训练循环
epochs = 200
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_mae = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs,adj)
        mse_loss = criterion_mse(outputs, targets)
        mae_loss = criterion_mae(outputs, targets)
        mse_loss.backward()
        optimizer.step()

        running_loss += mse_loss.item()
        running_mae += mae_loss.item()

    # 验证模型
    model.eval()
    val_mae = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs,adj)
            mae_loss = criterion_mae(outputs, targets)
            val_mae += mae_loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Train MAE: {running_mae/len(train_loader):.4f}, Test MAE: {val_mae/len(test_loader):.4f}")
    
    # 使用ReduceLROnPlateau来更新学习率
    scheduler.step(val_mae)

In [28]:
from pyplnoise import PinkNoise

X = np.load('X.npy')
X_noise = np.empty([1256,19,30000])

noise_gen = PinkNoise(250, 1, 100)

for i in range(1256):
    for j in range(19):
        coef = np.mean(X[i,j,:])*0.01
        X_noise[i,j,:] = X[i,j,:] + coef * noise_gen.get_series(30000).T


In [ ]:
import pandas as pd

# Convert the tensor to a pandas DataFrame
df_tensors = pd.DataFrame()
df_tensors['A'] = outputs.detach().cpu().numpy().squeeze()  # Using detach() to get a tensor that doesn't require gradients
df_tensors['B'] = targets.detach().cpu().numpy().squeeze() 
# Save the DataFrame to an Excel file
excel_path_specific_column = "pre.xlsx"
df_tensors.to_excel(excel_path_specific_column, index=False)

# Displaying the first few rows of the DataFrame for visualization
df_tensors.head()

#print(outputs.shape)